## Experiment to determine Frequency for running Model 

Author : Sai Siddartha Maram


Contact: msaisiddartha1@gmail.com | smaram_be16@thapar.edu

### Objectives
1. Run detection at Camera FPS, in our case the camera FPS is 30FPS, to start of I will run detection on every 30th image.
2. Log all the bounding boxes and classes of objects detected
3. Develop a certain metric in terms of centers of bounding boxes, classes detected to determine if the two scenes are different
4. Try delta methods and see if they match


In [1]:


%load_ext Cython
import cv2
import numpy as np
import argparse
import time
import cv2
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imutils

from scipy.spatial import distance as dist
import numpy as np

from skimage.measure import compare_ssim
from PIL import Image
import time

import scipy
import random

import tensorflow as tf
import math

# Image manipulation.
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
import inception5h
model = inception5h.Inception5h()

import imagehash
# My Packages
import constants
import helper



Instructions for updating:
Use tf.gfile.GFile.



Loading Model : YOLOV3

In [ ]:
options = constants.options_v3
np.random.seed(42)
LABELS = open(options['labelsPath']).read().strip().split("\n")
color = (255, 0, 0)
inception5h.maybe_download()

1. Load the darknet model using readNetFromDarkent(), feel free to try other DL models
2.  In CV2.dnn you load a model and then forward propgate the image for prediction, 
    the forwad function expects you give the layer till which you want the image to be 
    propgated, in our case since we want the final result, we want the image to be 
    propgated till the last layer. The .getLayerNames() gives the list of all layers.
    From here to identify the last layer, we need to get last unconnected layer, the 
    unconnected layers can be found by .getUnconnectedOutLayers().
3. blob, the cv.dnn does not take a raw image, instead takes a blob, A blob is an image which 
   is modified and smoothed as per the network requirments. For Ex. 
   YOLO takes in an image only of the size 416 x 416 (single channel)
   hence we reduce each color from [0-255] to [0-1] and image size (416,416)

In [ ]:
def predict_object(image_path,net):
    start = time.time()
    image = cv2.imread(image_path)
    #image=cv2.resize(image,(416,416))
    (H, W) = image.shape[:2]

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(
        image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    

    # show timing information on YOLO
    

    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > options['confidence']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, options['confidence'], options['threshold'])
    
    detections=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            temp_list=[]
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 3)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            plt.imshow(image)

            temp_list.append(x)
            temp_list.append(y)
            temp_list.append(w)
            temp_list.append(h)
            temp_list.append(LABELS[classIDs[i]])
            temp_list.append(confidences[i])
            detections.append(temp_list)
            
    print('Time Taken', time.time()-start)   
    return detections

In [ ]:
def predict_image(image,net):
    start = time.time()
    #image = cv2.imread(image_path)
    #image=cv2.resize(image,(416,416))
    (H, W) = image.shape[:2]

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(
        image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    

    # show timing information on YOLO
    

    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > options['confidence']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, options['confidence'], options['threshold'])
    
    detections=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            temp_list=[]
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 3)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            plt.imshow(image)

            temp_list.append(x)
            temp_list.append(y)
            temp_list.append(w)
            temp_list.append(h)
            temp_list.append(LABELS[classIDs[i]])
            temp_list.append(confidences[i])
            detections.append(temp_list)
            
    print('Time Taken', time.time()-start)   
    return detections

In [ ]:
net = cv2.dnn.readNetFromDarknet(options['model'], options['load'])

In [ ]:
detections=predict_object('testing_images/4561.jpg',net)
print('Number of Detections:', len(detections))
print(*detections,sep='\n')


In [ ]:
detections=predict_object('testing_images/16892.jpg',net)
print('Number of Detections:', len(detections))
print(*detections,sep='\n')


In [ ]:
def image_subtraction(path_image1,path_image2):
    img1 = cv2.imread(path_image1)
    img2 = cv2.imread(path_image2)


    start_time=time.time()
    grayA = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    (score, grad,diff) = compare_ssim(grayA, grayB, full=True,gradient=True)
    #value ranges between -1 to 1. 1 indicates a perfect match
    print('Time Taken for execution',time.time()-start_time)
    print("SSIM: {}".format(score))
    


In [ ]:
image_subtraction('testing_images/4561.jpg','testing_images/16892.jpg')

In [ ]:
def histogram_matching(path_image1, path_image2):
    img1 = cv2.imread(path_image1)
    img2 = cv2.imread(path_image2)
#     plt.hist(img1.ravel(),256,[0,256])
#     plt.show()
#     plt.hist(img2.ravel(),256,[0,256])
#     plt.show()

    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    start_time=time.time()

    hist1 = cv2.calcHist(img1, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()

    hist2 = cv2.calcHist(img2, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.normalize(hist2, hist2).flatten()

    #a = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT) ----> WASTE!
    #a = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL) ----->Waste!
    chi_square = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)
    print('[INFO]Time for Histogram Mapping',time.time()-start_time)

    return chi_square


In [ ]:
chi=histogram_matching('testing_images/4561.jpg','testing_images/4561.jpg')
print('CHI Square value',chi)

In [ ]:
chi=histogram_matching('testing_images/4561.jpg','testing_images/16892.jpg')
print('CHI Square value',chi)

In [ ]:
chi=histogram_matching('testing_images/4561.jpg','testing_images/2.jpg')
print('CHI Square value',chi)

### Learnable Encryption

In [ ]:
img=cv2.imread('testing_images/16892.jpg')
print(img.shape)

In [ ]:
N=8
M=8
plt.imshow(img)

In [ ]:
# b=[]
# for x in range(0,img.shape[0],M):
#     for y in range(0,img.shape[1],N):
#         b.append(img[x:x+M,y:y+N])
# print(len(b))

blank_image = np. zeros(shape=[1080,1920, 3], dtype=np. uint8)

for x in range(0,img.shape[0],M):
    for y in range(0,img.shape[1],N):
        blank_image[x:x+M,y:y+N]=img[x:x+M,y:y+N]+129

cv2.imshow('img',blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
plt.imshow(blank_image)

In [ ]:
detections=predict_image(blank_image,net)
print('Number of Detections:', len(detections))
print(*detections,sep='\n')


### Look up table for color mapping

In [12]:
def generate_new_look_up_table_random(R,n):
    R_new=[R[i:i + n] for i in range(0, len(R), n)]
    res = random.sample(R_new, len(R_new)) 
    R_prime=[]
    for i in res:
        for j in i:
            R_prime.append(j)
 
    return R_prime

In [16]:
%%cython -a
cimport numpy


#@cython.boundscheck(False)
cpdef unsigned char[:,:, :] decoder_fast(numpy.ndarray lut, unsigned char [:,:,:] im_encoded):
    # set the variable extension types
    cdef int i, j, w, h
    cdef list R = lut[:,0,0].tolist()
    cdef list G = lut[:,0,1].tolist()
    cdef list B = lut[:,0,2].tolist()
    
    h = im_encoded.shape[0]
    w = im_encoded.shape[1]

    for i in range(0, h):
        for j in range(0, w):
            im_encoded[i][j][0]=R.index(im_encoded[i][j][0])
            im_encoded[i][j][1]=G.index(im_encoded[i][j][1])
            im_encoded[i][j][2]=B.index(im_encoded[i][j][2])
    print(im_encoded)
    return im_encoded

In [13]:
def lut_generator(n):
    lut = np.zeros((256, 1, 3), dtype=np.uint8)
    lut[:, 0, 0] = generate_new_look_up_table_random([i for i in range(0,256)],n)
    lut[:, 0, 1] = generate_new_look_up_table_random([i for i in range(0,256)],n)
    lut[:, 0, 2] = generate_new_look_up_table_random([i for i in range(0,256)],n)
    return lut

In [14]:
im = cv2.imread('testing_images/16892.jpg')

look_up_table=lut_generator(8)
im_color = cv2.LUT(im,look_up_table)


cv2.imshow('encoded',im_color)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [15]:
%reload_ext cython

In [17]:
image=decoder_fast(look_up_table,im_color)


<MemoryView of 'ndarray' object>


In [18]:
decrypted_image=np.asarray(image)
cv2.imshow('decrypted',decrypted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
detections=predict_object('testing_images/16892.jpg',net)
print('Number of Detections:', len(detections))
print(*detections,sep='\n')

In [ ]:
detections=predict_image(im_color,net)
print('Number of Detections:', len(detections))
print(*detections,sep='\n')

In [ ]:
cv2.imshow('encoded',im_color)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Detection Performance and Evaluation Based on Image Sizes

In [ ]:
def multi_shape_detection(image_path,net):
    img=cv2.imread(image_path)
    
    h,w,_=img.shape
    #half
    print(img.shape)
    img_1 = cv2.resize(img,(w//2,h//2))
    print(img_1.shape)
    #one fourth
    img_2 = cv2.resize(img_1,(w//4,h//4))
    print(img_2.shape)
    detections = predict_image(img,net)
    detection_half = predict_image(img_1,net)
    detection_quarter = predict_image(img_2,net)
    
    return detections,detection_half,detection_quarter

In [ ]:
image_path='testing_images/16892.jpg' 
detections,detection_half,detections_quarter = multi_shape_detection(image_path,net)
# print('Number of Detections in full image:', len(detections))
# print(*detections,sep='\n')

In [ ]:
print('Number of Detections in half size:', len(detection_half))
print(*detection_half,sep='\n')

In [ ]:
print('Number of Detections in quarter size:', len(detections_quarter))
print(*detections_quarter,sep='\n')